# Script 1 — Configuração Geral e Importações  
Carrega as bibliotecas e define os caminhos dos arquivos.  
Tudo que o projeto usa começa daqui.

In [1]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path

# Configuração de visualização
pd.set_option('display.max_columns', None)

# --- DEFINIÇÃO DE CAMINHOS INTELIGENTE ---
# Detecta automaticamente onde este script está rodando
if '__file__' in globals():
    BASE_DIR = Path(__file__).parent 
else:
    BASE_DIR = Path.cwd() # Caso esteja rodando no Jupyter

# Estrutura esperada: app/data/raw e app/data/processed
# Se não achar, tenta procurar a partir da raiz
if (BASE_DIR / "app" / "data").exists():
    DATA_DIR = BASE_DIR / "app" / "data"
else:
    # Fallback: Assume que estamos na raiz e a pasta data está em ./app/data
    DATA_DIR = BASE_DIR / "app" / "data"

DIR_RAW = DATA_DIR / "raw"
DIR_PROCESSED = DATA_DIR / "processed"

# Garante que a pasta de saída existe
os.makedirs(DIR_PROCESSED, exist_ok=True)

print(f"📂 Diretório RAW: {DIR_RAW}")
print(f"💾 Diretório PROCESSED: {DIR_PROCESSED}")

# Caminhos dos arquivos
path_telemetria = DIR_RAW / "telemetria_detalhada_30dias.csv"
path_producao   = DIR_RAW / "historico_producao_1ano.csv"
path_eventos    = DIR_RAW / "eventos_industriais.csv"

# --- VALIDAÇÃO INICIAL ---
print("\n🔍 Verificando arquivos de entrada...")
arquivos_necessarios = [
    ("Telemetria", path_telemetria),
    ("Produção", path_producao),
    ("Eventos", path_eventos)
]

arquivos_faltando = []
for nome, caminho in arquivos_necessarios:
    if caminho.exists():
        print(f"  ✅ {nome}: OK")
    else:
        print(f"  ❌ {nome}: NÃO ENCONTRADO em {caminho}")
        arquivos_faltando.append(nome)

if arquivos_faltando:
    print(f"\n⚠️  ATENÇÃO CRÍTICA: Faltam arquivos! Execute o simulador primeiro.")
    print(f"   Script: simulador_industrial_hibrido.py")
    # Não para o script aqui para permitir depuração, mas avisa forte

📂 Diretório RAW: c:\Users\anaca\OneDrive\Área de Trabalho\projetosenai\app\data\raw
💾 Diretório PROCESSED: c:\Users\anaca\OneDrive\Área de Trabalho\projetosenai\app\data\processed

🔍 Verificando arquivos de entrada...
  ✅ Telemetria: OK
  ✅ Produção: OK
  ✅ Eventos: OK


# Script 2 — Processamento da Telemetria  
Aqui faço a limpeza das leituras da máquina, padronizo temperatura, arrumo datas  
e gero a versão “Silver” dos dados de telemetria.

In [2]:
print("\n📥 Processando Telemetria (Sensor Data)...")

# 1. Ingestão com tratamento de erro
try:
    telemetria_df = pd.read_csv(path_telemetria)
    print(f"  📊 Linhas carregadas: {len(telemetria_df):,}")
except FileNotFoundError:
    print("❌ Erro Crítico: Arquivo de telemetria não encontrado.")
    # Cria dataframe vazio para não quebrar o resto
    telemetria_df = pd.DataFrame(columns=["timestamp", "maquina_id", "temp_matriz_c", "pressao_mpa", "umidade_pct", "ciclo_tempo_s"])

if not telemetria_df.empty:
    # 2. Tratamento de Data
    telemetria_df["timestamp"] = pd.to_datetime(telemetria_df["timestamp"], errors="coerce")

    # 3. Função de Limpeza de Temperatura (Regex)
    def clean_temp(x):
        if pd.isna(x): return None
        x_str = re.sub(r"[^0-9\.,-]", "", str(x))
        x_str = x_str.replace(",", ".") 
        try:
            return float(x_str)
        except:
            return None

    # Aplica a limpeza
    telemetria_df["temp_matriz_c"] = telemetria_df["temp_matriz_c"].apply(clean_temp)

    # 4. Inputação (Forward Fill)
    telemetria_df["temp_matriz_c"] = telemetria_df["temp_matriz_c"].ffill()
    
    # Segurança: Se ainda houver NaN (no início), preenche com a média
    if telemetria_df["temp_matriz_c"].isna().sum() > 0:
        media_temp = telemetria_df["temp_matriz_c"].mean()
        telemetria_df["temp_matriz_c"] = telemetria_df["temp_matriz_c"].fillna(media_temp)
        print(f"  ⚠️  Valores iniciais nulos preenchidos com a média ({media_temp:.1f}°C)")

    # 5. Garantir Tipos Numéricos
    cols_float = ["pressao_mpa", "umidade_pct", "ciclo_tempo_s"]
    for col in cols_float:
        if col in telemetria_df.columns:
            telemetria_df[col] = pd.to_numeric(telemetria_df[col], errors="coerce")

    # 6. Ordenação e Salvamento
    telemetria_df = telemetria_df.sort_values(["maquina_id", "timestamp"]).reset_index(drop=True)

    save_path = DIR_PROCESSED / "telemetria_silver.csv"
    telemetria_df.to_csv(save_path, index=False)

    print(f"✅ Telemetria salva em: {save_path}")
    print("\n📈 Relatório de Qualidade:")
    print(f"  • Período: {telemetria_df['timestamp'].min()} a {telemetria_df['timestamp'].max()}")
    print(f"  • Máquinas: {telemetria_df['maquina_id'].nunique()}")
    print(f"  • Taxa de Defeitos Global: {telemetria_df['flag_defeito'].mean()*100:.2f}%")


📥 Processando Telemetria (Sensor Data)...
  📊 Linhas carregadas: 648,000
✅ Telemetria salva em: c:\Users\anaca\OneDrive\Área de Trabalho\projetosenai\app\data\processed\telemetria_silver.csv

📈 Relatório de Qualidade:
  • Período: 2025-10-24 05:00:00.961210 a 2025-11-23 01:59:54.961210
  • Máquinas: 2
  • Taxa de Defeitos Global: 47.21%


# Script 3 — Processamento da Produção  
Calculo taxa de refugo, padronizo sinalizações, arrumo datas  
e gero a versão “Silver” dos dados de produção.

In [3]:
print("\n📥 Processando Histórico de Produção...")

try:
    producao_df = pd.read_csv(path_producao)
    print(f"  📊 Linhas carregadas: {len(producao_df):,}")
except FileNotFoundError:
    print("❌ Erro Crítico: Arquivo de produção não encontrado.")
    producao_df = pd.DataFrame()

if not producao_df.empty:
    # 2. Tratamento de Data
    producao_df["timestamp"] = pd.to_datetime(producao_df["timestamp"], errors="coerce")

    # 3. Engenharia de Atributos (KPIs)
    # Taxa de Refugo (tratando divisão por zero)
    producao_df["taxa_refugo"] = producao_df["pecas_refugadas"] / producao_df["pecas_produzidas"]
    producao_df["taxa_refugo"] = producao_df["taxa_refugo"].replace([np.inf, -np.inf], 0).fillna(0)

    # Flag binária: Houve produção?
    producao_df["produziu_algo"] = (producao_df["pecas_produzidas"] > 0).astype(int)

    # Custo estimado (Configurável)
    TARIFA_ENERGIA = 0.80 # R$/kWh
    producao_df["custo_energia_estimado"] = producao_df["consumo_kwh"] * TARIFA_ENERGIA
    
    # Dados Sazonais (Para análise futura)
    producao_df["mes"] = producao_df["timestamp"].dt.month
    producao_df["dia_semana"] = producao_df["timestamp"].dt.dayofweek # 0=Seg, 6=Dom

    # 4. Ordenação e Salvamento
    producao_df = producao_df.sort_values(["maquina_id", "timestamp"]).reset_index(drop=True)

    save_path_prod = DIR_PROCESSED / "producao_silver.csv"
    producao_df.to_csv(save_path_prod, index=False)

    print(f"✅ Produção salva em: {save_path_prod}")
    print("\n📈 Resumo Financeiro/Produção:")
    print(f"  • Total Produzido: {producao_df['pecas_produzidas'].sum():,} peças")
    print(f"  • Custo Energia Total: R$ {producao_df['custo_energia_estimado'].sum():,.2f}")
    print(f"  • Taxa Média Refugo: {producao_df['taxa_refugo'].mean()*100:.2f}%")


📥 Processando Histórico de Produção...
  📊 Linhas carregadas: 17,522
✅ Produção salva em: c:\Users\anaca\OneDrive\Área de Trabalho\projetosenai\app\data\processed\producao_silver.csv

📈 Resumo Financeiro/Produção:
  • Total Produzido: 13,629,627 peças
  • Custo Energia Total: R$ 168,797.30
  • Taxa Média Refugo: 2.10%


# Script 4 — Processamento dos Eventos Industriais  
Padronizo o texto dos eventos, converto severidade para número  
e gero a versão “Silver” dos eventos da fábrica.

In [4]:
print("\n📥 Processando Eventos e Alarmes...")

try:
    eventos_df = pd.read_csv(path_eventos)
    print(f"  📊 Eventos carregados: {len(eventos_df):,}")
except FileNotFoundError:
    print("❌ Erro Crítico: Arquivo de eventos não encontrado.")
    eventos_df = pd.DataFrame()

if not eventos_df.empty:
    # 2. Tratamento de Data
    eventos_df["timestamp"] = pd.to_datetime(eventos_df["timestamp"], errors="coerce")

    # 3. Padronização de Texto
    cols_texto = ["evento", "severidade", "origem"]
    for col in cols_texto:
        if col in eventos_df.columns:
            eventos_df[col] = eventos_df[col].astype(str).str.strip().str.capitalize()

    # 4. Mapeamento de Severidade
    map_sev = {"Baixa": 1, "Média": 2, "Alta": 3}
    eventos_df["sev_codigo"] = eventos_df["severidade"].map(map_sev).fillna(0).astype(int)

    # 5. Garantir ID único (Segurança)
    if "evento_id" in eventos_df.columns:
        eventos_df["evento_id"] = eventos_df["evento_id"].str.strip().str.upper()
    else:
        # Cria IDs se não existirem (Fallback)
        eventos_df["evento_id"] = [f"EVT-AUTO-{i+1:04d}" for i in range(len(eventos_df))]
        print("  ⚠️  Coluna 'evento_id' não encontrada. IDs gerados automaticamente.")

    # 6. Ordenação e Salvamento
    eventos_df = eventos_df.sort_values(["timestamp"]).reset_index(drop=True)

    save_path_evt = DIR_PROCESSED / "eventos_silver.csv"
    eventos_df.to_csv(save_path_evt, index=False)

    print(f"✅ Eventos salvos em: {save_path_evt}")
    
    print("\n📊 Top 5 Problemas Mais Frequentes (Pareto):")
    top_problemas = eventos_df["evento"].value_counts().head(5)
    for problema, qtd in top_problemas.items():
        print(f"  • {problema}: {qtd} ocorrências")


📥 Processando Eventos e Alarmes...
  📊 Eventos carregados: 200
✅ Eventos salvos em: c:\Users\anaca\OneDrive\Área de Trabalho\projetosenai\app\data\processed\eventos_silver.csv

📊 Top 5 Problemas Mais Frequentes (Pareto):
  • Falha de pressão: 45 ocorrências
  • Baixa umidade: 43 ocorrências
  • Falha de motor: 40 ocorrências
  • Sobrecarga elétrica: 37 ocorrências
  • Parada programada: 35 ocorrências
